In [1]:
import os
import json

from tqdm.notebook import tqdm

from react_agent.src.config.system_parameters import CodebaseSearcherSettings

from react_agent.src.agent_tools.source_code_retriever import SourceCodeRetriever

from react_agent.src.util.abap_repository import ABAPClassRepository
from react_agent.src.util.memory_manager import MemoryManager
from react_agent.src.util.code_summarizer import CodeSummarizer

In [2]:
notebook_dir = os.path.abspath("")

file_path = os.path.abspath(os.path.join(notebook_dir, "./", "resources", "temp.json"))

with open(file_path, encoding="utf8") as f:
    json_file = json.load(f)

In [3]:
classes: list[tuple] = []

for my_class in json_file["classes"]:
    classes.append((my_class["name"].lower(), my_class["description"]))

In [4]:
CODE_SEARCHER_SETTINGS = CodebaseSearcherSettings()

MEMORY_MANAGER = MemoryManager(
    memory_store_type="Postgres",
    namespace=CODE_SEARCHER_SETTINGS.namespace,
    embedding_fields=["description"],
)

In [5]:
code_lookup = SourceCodeRetriever()

for my_class in tqdm(classes, desc="Processing Classes"):
    print("Class name: ", my_class[0])
    class_name = my_class[0]
    class_description = my_class[1]

    class_code = code_lookup._run(class_name=class_name)

    print("\t- Started indexing")
    repository = ABAPClassRepository(source_code=class_code)
    if len(repository.classes) == 0:
        print(f"\t- No methods found for class: {class_name}")
        continue

    print(
        f"\t- Finished indexing, number of methods: {len(repository.classes.get(class_name).keys())}"
    )

    for method_name in repository.classes[class_name].keys():
        if not MEMORY_MANAGER.is_memory_present(f"{class_name}.{method_name}"):
            print(f"\t- Summarizing and adding method: {method_name}")
            # Summarize the code of the method and add it to the memory
            memory_content = {
                "description": CodeSummarizer.summarize_code(
                    f"Class description: {class_description}\n"
                    + repository.get_content_by_class_and_method(
                        class_name=class_name, method_name=method_name
                    )
                ),
                "code": repository.get_content_by_class_and_method(
                    class_name=class_name, method_name=method_name
                ),
            }
            MEMORY_MANAGER.add_memory(
                memory_title=f"{class_name}.{method_name}",
                memory_content=memory_content,
            )
        else:
            print(f"\t- Skipping method: {method_name}")
            continue

Processing Classes:   0%|          | 0/3 [00:00<?, ?it/s]

Class name:  cl_edoc_map_co_sd_ubl_21
	- Started indexing
	- Finished indexing, number of methods: 16
	- Summarizing and adding method: fill_allowance_charge
	- Summarizing and adding method: fill_credit_note_line
	- Summarizing and adding method: fill_cr_note
	- Summarizing and adding method: fill_customization_id
	- Summarizing and adding method: fill_debit_note_line
	- Summarizing and adding method: fill_de_note
	- Summarizing and adding method: fill_document_line
	- Summarizing and adding method: fill_invoice
	- Summarizing and adding method: fill_invoice_line
	- Summarizing and adding method: fill_item_identification
	- Summarizing and adding method: fill_monetary_total
	- Summarizing and adding method: fill_tax_total
	- Summarizing and adding method: fill_withholding_tax_total
	- Summarizing and adding method: get_line_count
	- Summarizing and adding method: get_tax_amounts
	- Summarizing and adding method: init_mapping
Class name:  cl_edoc_int_con_cl_siimail_pul
	- Started index